# Lab Assignment Six: Convolutional Network Architectures
by:
* Hayden Donofrio
* Riley Bates
* Andrew Breslauer
* Chandler Choate

## Section 1: Preparation

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.layers import average 
from keras.models import Input, Model
from sklearn import metrics as mt
from matplotlib import pyplot as plt
from skimage.io import imshow
import seaborn as sns
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [8]:
pneumonia_images = []
healthy_images = []
import os
import struct
import numpy as np
import glob
import cv2
images = glob.glob("chest_xray/train/NORMAL/*")
for image in images:
    im = cv2.imread(image,0)
    im = cv2.resize(im,(200,200))
    im = im.flatten()/255 -.5
    healthy_images.append(im)
images = glob.glob("chest_xray/train/PNEUMONIA/*")
for image in images:
    im = cv2.imread(image,0)
    im = cv2.resize(im,(200,200))
    im = im.flatten()/255 -.5
    pneumonia_images.append(im)


In [9]:
#Now lets put them together with y values
healthy_images
b = np.zeros((len(healthy_images),1))
a = np.ones((len(pneumonia_images),1))
healthy_images = np.hstack((healthy_images, b))
pneumonia_images = np.hstack((pneumonia_images, a))
images = np.vstack([healthy_images, pneumonia_images])

print(len(healthy_images))
print(len(pneumonia_images))

1341
3875


In [42]:
y = images[:,-1]
X = images[:,:-1]

# folding below
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)
img_wh = 200
y_test_ohe = keras.utils.to_categorical(y_test, 2)
y_train_ohe = keras.utils.to_categorical(y_train, 2)
X_train_img = np.expand_dims(X_train.reshape((-1,img_wh,img_wh)), axis=3)
X_test_img = np.expand_dims(X_test.reshape((-1,img_wh,img_wh)), axis=3)
folds = StratifiedKFold(n_splits=5,random_state=1).split(X_train, y_train)

## Train/Test Split Method

For our dataset, we implemented 5 stratified KFolds for testing and training. We have limited data to train and test on, so using stratified folds is crucial to make sure that the model does not overtrain to a specific dataset. We believe that 5 folds is a good balance of computation time and ensuring enough variability in the data.

If this model were used in practice, it would not have to be trained too frequently, depending on if it were implemented in a single doctor's office or if it were a shared resource between hospitals or something along these lines. Pneumonia is most common in elderly individuals, and is usually caused by a viral infection or other inflammation in the lungs, which are both conditions that occur prior to pneumonia, so the diagnosed cases of pneumonia are not as high as something such as the flu. 

Since we wouldn't have to train the model too frequently in actual implementation, we feel that 5 fold cross validation is the best method of splitting up our training and test data.

## Evaluation Metric

We will use precision score as our evaluation matrix since we care mostly about identifying who has pneumonia. Identifying positives or negatives for a sickness is similar to the imbalanced classification problem that occurs in disease detection: where the rate of positives (individuals that are sick) is very low. We have two classes to identify - those that have pneumonia and those who do not, but one category represents a large amount of data points. This is an indiciation that accuracy is clearly not a good measure for assessing performance.

We feel that precision score is a valuable metric to use because in our classification task, the most important thing is correctly identifying true positives, and we want to measure our success against true and false positives, instead of true positives and false negatives with recall score.

## Section 2: Modeling

In [43]:

#this is expansion on the dataset. This is a slower way to do it. When we use a keras generator we call .fit on a python generator
#this will yield batches

datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False, #do we want to make it 0 mean
    featurewise_std_normalization=False, #do we take the whole batch and make it 0 mean, no
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=5, # used, Int. Degree range for random rotations. Randomly rotate images 5 degrees 
    width_shift_range=0.1, # used, Float (fraction of total width). Range for random horizontal shifts. 
    height_shift_range=0.1, # used,  Float (fraction of total height). Range for random vertical shifts.
    shear_range=0., # Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=False, 
    rescale=None) #this generator will esentially run forever. This will manipulate our data, will give us different datasets every time


Expansion explained

## AlexNet Style CNN

In [44]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)
#Instantiate an empty model
#get a bigger dataset
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=32, input_shape=(200,200,1), kernel_size=(14,14),  padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=64, kernel_size=(7,7), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid', data_format="channels_last"))

# 3rd Convolutional Layer
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='valid', data_format="channels_last"))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', data_format="channels_last"))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='valid', data_format="channels_last"))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 187, 187, 32)      6304      
_________________________________________________________________
activation_28 (Activation)   (None, 187, 187, 32)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 93, 93, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 87, 87, 64)        100416    
_________________________________________________________________
activation_29 (Activation)   (None, 87, 87, 64)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 41, 41, 128)      

In [46]:
scores = []
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('alexNet_log.csv', append=True, separator=';')
histories= []
#datagen.fit(X_train)
print(folds)
for k, (train, test) in enumerate(folds):
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=[keras.metrics.Recall()])
    print("new fold")
    datagen.fit(X_train_img)
    histories.append(model.fit_generator(datagen.flow(X_train_img[train], y_train_ohe[train], batch_size=32),
             steps_per_epoch=int(len(X_train)/2),
             epochs=10, verbose=1,
             validation_data=(X_test_img,y_test_ohe),
             callbacks=[EarlyStopping(monitor='val_loss', patience=2)]
             ))


<generator object _BaseKFold.split at 0x0000026DAE36A1C8>
new fold
Epoch 1/10
166/166 [==============================] - 1154s 7s/step - loss: 0.6736 - recall_12: 0.6169 - val_loss: 0.6500 - val_recall_12: 0.6560
Epoch 2/10
166/166 [==============================] - 1147s 7s/step - loss: 0.6670 - recall_12: 0.6170 - val_loss: 0.6505 - val_recall_12: 0.6560
Epoch 3/10
166/166 [==============================] - 1147s 7s/step - loss: 0.6678 - recall_12: 0.6168 - val_loss: 0.6490 - val_recall_12: 0.6560
Epoch 4/10
166/166 [==============================] - 1148s 7s/step - loss: 0.6670 - recall_12: 0.6166 - val_loss: 0.6489 - val_recall_12: 0.6560
Epoch 5/10
166/166 [==============================] - 1147s 7s/step - loss: 0.6679 - recall_12: 0.6170 - val_loss: 0.6512 - val_recall_12: 0.6560
Epoch 6/10
166/166 [==============================] - 1149s 7s/step - loss: 0.6668 - recall_12: 0.6169 - val_loss: 0.6488 - val_recall_12: 0.6560
Epoch 7/10
166/166 [==============================] - 114

KeyError: 'recall_12'

In [31]:
old_histories = histories

In [30]:
histories

In [32]:
old_histories["loss"]

In [47]:
old_histories[0].history

{'val_loss': [0.3086502356529236,
  0.3283050652742386,
  0.292212664604187,
  0.2826671036481857,
  0.23993148624897004,
  0.2301979398727417,
  0.2715948328971863,
  0.4862943035364151],
 'val_recall_6': [0.9039999842643738,
  0.8880000114440918,
  0.9200000166893005,
  0.871999979019165,
  0.8880000114440918,
  0.9039999842643738,
  0.8960000276565552,
  0.8640000224113464],
 'loss': [0.268541384109937,
  0.20544711815774755,
  0.19613678438893906,
  0.18069041215864842,
  0.17609683662239048,
  0.15704124928178156,
  0.14645729406746646,
  0.11541567897236543],
 'recall_6': [0.8964497,
  0.92228836,
  0.9276792,
  0.93180305,
  0.9381614,
  0.94183004,
  0.9490741,
  0.9566799]}

In [48]:
test = old_histories

In [49]:
test

In [51]:
with open("alexnet.txt", "w") as f:
    for history in test:
        print(history.history, file=f)

## ResNet Style CNN

In [ ]:
# Andrew's stuff here

## Performance Visualization

## Comparison to MLP

In [ ]:
%%time

y_train_ohe = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test_ohe = keras.utils.to_categorical(y_test, NUM_CLASSES)

# make a 3 layer keras MLP
mlp = Sequential()
mlp.add( Dense(input_dim=X_train.shape[1], units=30, activation='relu') )
mlp.add( Dense(units=15, activation='relu') )
mlp.add( Dense(NUM_CLASSES) )
mlp.add( Activation('softmax') )

mlp.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

mlp.fit(X_train, y_train_ohe, 
        batch_size=32, epochs=150, 
        shuffle=True, verbose=0)

## Section 3: Exceptional Work

Our group has turned in Lab 6 by Sunday, December 1 for the exceptional work credit.